# Falcon Baseline (no RAG)

In [ ]:
import requests
import json
from tqdm import tqdm
import csv
import re
url = 'https://a7tncd274b.execute-api.eu-west-3.amazonaws.com/InitialStage/Falcon7B'
# Set the appropriate headers for a JSON payload
headers = {
    "Content-Type": "application/json"
}


In [ ]:
def create_prompt(question, options):
    options_text = "\n".join([f"Option {chr(65+i)}: {opt}" for i, opt in enumerate(options)])
    prompt = (
        f"Instruct: You will answer each question correctly by giving only the Option ID, the letter that follows each Option.\n"
        f"The output should be in the format: Option <Option id>\n\n"
        f"Here is an example of a question and its expected answer:\n\n"
        f"Question: When are devices required to send the GTS Request command?\n"
        f"Options:\n"
        f"Option A: Only devices without a short address\n"
        f"Option B: Only devices using extended addressing\n"
        f"Option C: Only devices capable of sending it\n"
        f"Option D: All devices\n"
        f"Option E: None of the above\n"
        f"Answer: Option D\n\n"
        f"Provide the answer to the following multiple choice question.\n"
        f"Question: {question}\n"
        f"Options:\n{options_text}\n"
        f"Answer:"
    )
    return prompt

In [ ]:
def get_api_response(prompt):
    data = {
        "inputs": prompt,
        "parameters": {
            "max_new_tokens": 20,
            "return_full_text": False,
            "do_sample": False,
            "top_k": 1,
            "stop": ["Output:"]
        }
    }
    json_data = json.dumps(data)
    try:
        response = requests.post(url, data=json_data, headers=headers)
        if response.status_code == 200:
            return response.text
        else:
            return f"Error: {response.status_code}"
    except Exception as e:
        return f"An error occurred: {e}"

In [ ]:
# Read questions from the JSON file
with open('TeleQnA_testing1.txt', 'r') as file1:
  with open('questions_new.txt', 'r') as file2:
    questions = json.load(file1)
    questions.update(json.load(file2))

In [ ]:
questions = dict(list(questions.items())[1810:])

In [ ]:
responses = []

# Loop through each question and get the response
for q_id, q_data in tqdm(questions.items(), desc="Processing questions"):
    q_id_number = q_id.split()[1]
    question_text = q_data["question"]
    question_text = re.sub(r'\s*\[.*?\]\s*$', '', question_text)
    options = [v for k, v in q_data.items() if k.startswith("option")]
    prompt = create_prompt(question_text, options)
    response = get_api_response(prompt)
    response = json.loads(json.loads(response)["body"])[0]["generated_text"]
    print(f"\nPrompt:\n{prompt}\n{response}")

    # Create a mapping of letters to numbers
    letter_to_number = {chr(65 + i): i + 1 for i in range(26)}

    # Extract the answer ID from the response
    match = re.search(r'Option (\b[A-J]\b)', response)
    if match:
        try:
            answer_id = match.group(1)
            print(f"Answer ID: {letter_to_number[answer_id]}")
            responses.append([q_id_number, letter_to_number[answer_id], "Falcon 7.5B"])
        except (KeyError, IndexError) as e:
            responses.append([q_id_number, "Error", "Falcon 7.5B"])
            print(f"Error processing question {q_id}: {response}")
    else:
        # Check if the response is one of the options
        option_found = False
        for idx, option in enumerate(options):
            if option in response:
                print(f"Matched option in response: {option} to {idx}")
                responses.append([q_id_number, idx + 1, "Falcon 7.5B"])
                option_found = True
                break

        if not option_found:
            responses.append([q_id_number, "Error", "Falcon 7.5B"])
            print(f"Error processing question {q_id_number}: {response}")

# Save responses to a CSV file
with open('output_results.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(["Question_ID", "Answer_ID", "Task"])
    csvwriter.writerows(responses)

print("Processing complete. Responses saved to 'output_results.csv'.")


Processing questions:   0%|          | 1/2366 [00:01<42:51,  1.09s/it]


Prompt:
Instruct: You will answer each question correctly by giving only the Option ID, the letter that follows each Option.
The output should be in the format: Option <Option id>

Here is an example of a question and its expected answer:

Question: When are devices required to send the GTS Request command?
Options:
Option A: Only devices without a short address
Option B: Only devices using extended addressing
Option C: Only devices capable of sending it
Option D: All devices
Option E: None of the above
Answer: Option D

Provide the answer to the following multiple choice question.
Question: When can a gNB transmit a DL transmission(s) on a channel after initiating a channel occupancy?
Options:
Option A: Regardless of the duration of the gap between the DL transmission(s) and any previous transmission(s) corresponding to the channel occupancy initiated by the gNB.
Option B: If the gap between the DL transmission(s) and any previous transmission(s) corresponding to the channel occupanc

Processing questions:   0%|          | 2/2366 [00:01<31:59,  1.23it/s]


Prompt:
Instruct: You will answer each question correctly by giving only the Option ID, the letter that follows each Option.
The output should be in the format: Option <Option id>

Here is an example of a question and its expected answer:

Question: When are devices required to send the GTS Request command?
Options:
Option A: Only devices without a short address
Option B: Only devices using extended addressing
Option C: Only devices capable of sending it
Option D: All devices
Option E: None of the above
Answer: Option D

Provide the answer to the following multiple choice question.
Question: What does OTA REFSENS requirement ensure?
Options:
Option A: The accuracy of the receiver in filtering out out-of-band signals.
Option B: The ability of the receiver to receive a wanted signal in the presence of an interfering signal.
Option C: The receiver's ability to receive an unwanted signal in the presence of a wanted signal.
Option D: The accuracy of the receiver in filtering out adjacent c

Processing questions:   0%|          | 3/2366 [00:02<28:40,  1.37it/s]

...


Prompt:
Instruct: You will answer each question correctly by giving only the Option ID, the letter that follows each Option.
The output should be in the format: Option <Option id>

Here is an example of a question and its expected answer:

Question: When are devices required to send the GTS Request command?
Options:
Option A: Only devices without a short address
Option B: Only devices using extended addressing
Option C: Only devices capable of sending it
Option D: All devices
Option E: None of the above
Answer: Option D

Provide the answer to the following multiple choice question.
Question: Which set of analytics is not relevant for criteria at the Network Slice level?
Options:
Option A: Network Performance information
Option B: NF Load information
Option C: UE mobility information
Option D: Expected UE behavioural parameters
Option E: UE Communication information
Answer:
 Option A
Answer ID: 1


Processing questions: 100%|█████████▉| 2365/2366 [25:07<00:00,  1.61it/s]


Prompt:
Instruct: You will answer each question correctly by giving only the Option ID, the letter that follows each Option.
The output should be in the format: Option <Option id>

Here is an example of a question and its expected answer:

Question: When are devices required to send the GTS Request command?
Options:
Option A: Only devices without a short address
Option B: Only devices using extended addressing
Option C: Only devices capable of sending it
Option D: All devices
Option E: None of the above
Answer: Option D

Provide the answer to the following multiple choice question.
Question: Which signature algorithm is used for authenticating the MCData payloads in MCData?
Options:
Option A: DP_MD5
Option B: DP_RSA_SHA256
Option C: HMAC_SHA1
Option D: DP_ECCSI
Option E: DP_AES_256_GCM
Answer:
 Option B
Answer ID: 2


Processing questions: 100%|██████████| 2366/2366 [25:07<00:00,  1.57it/s]


Prompt:
Instruct: You will answer each question correctly by giving only the Option ID, the letter that follows each Option.
The output should be in the format: Option <Option id>

Here is an example of a question and its expected answer:

Question: When are devices required to send the GTS Request command?
Options:
Option A: Only devices without a short address
Option B: Only devices using extended addressing
Option C: Only devices capable of sending it
Option D: All devices
Option E: None of the above
Answer: Option D

Provide the answer to the following multiple choice question.
Question: Which signature method for XML HMAC signature shall be supported?
Options:
Option A: HMAC-SHA1
Option B: HMAC-MD5
Option C: RSA-SHA256
Option D: DSA-SHA1
Option E: HMAC-SHA256
Answer:
 Option B
Answer ID: 2
Processing complete. Responses saved to 'output_results.csv'.
